In [3]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [4]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,Covid-19 News: Live Updates The New York Times,https://www.nytimes.com/2020/09/13/world/coron...,"Sun, 13 Sep 2020 19:52:30 GMT",2020-09-13 19:52:30+00:00
7,Ohio State University canceled spring break 20...,https://www.cnn.com/2020/09/13/us/ohio-state-s...,"Sun, 13 Sep 2020 17:01:00 GMT",2020-09-13 17:01:00+00:00
5,Coronavirus in Minnesota: state seeing deadlie...,https://www.minnpost.com/health/2020/09/the-da...,"Sun, 13 Sep 2020 16:49:02 GMT",2020-09-13 16:49:02+00:00
6,Latest on COVID-19 in MN: New cases trend down...,https://www.mprnews.org/story/2020/09/13/lates...,"Sun, 13 Sep 2020 16:15:00 GMT",2020-09-13 16:15:00+00:00
2,Governor Cuomo Updates New Yorkers on State's ...,https://www.governor.ny.gov/news/governor-cuom...,"Sun, 13 Sep 2020 15:04:32 GMT",2020-09-13 15:04:32+00:00
9,"Redefining Covid-19: Months after infection, p...",https://www.cnn.com/2020/09/13/health/long-hau...,"Sun, 13 Sep 2020 14:09:00 GMT",2020-09-13 14:09:00+00:00
1,COVID-19 Daily Update 9-13-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sun, 13 Sep 2020 14:03:19 GMT",2020-09-13 14:03:19+00:00
3,COVID-19 Data Review Panel Examines State's Da...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Sat, 12 Sep 2020 21:19:52 GMT",2020-09-12 21:19:52+00:00
8,Pfizer and BioNTech announce plan to expand Co...,https://www.statnews.com/2020/09/12/pfizer-and...,"Sat, 12 Sep 2020 15:00:50 GMT",2020-09-12 15:00:50+00:00
4,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Fri, 11 Sep 2020 21:30:00 GMT",2020-09-11 21:30:00+00:00
